#BERTopic on Quran Urdu Translations
In this notebook I have implemented Topic Modeling on Urdu translations based topic modelling technique BERTopic.

Shaista Zulfiqar

## Mounting Google Drive
If the dataset is on Google Drive then you have to mount over google drive with collaboratory.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive




#Installing required dependencies
**One thing to remember is that after installing libraries you have to restart the run time again so that other dependencies are not affected by it.**

In [ ]:
!pip install bertopic
!pip install -U sentence-transformers
!pip install urduhack

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 49.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.7/85.7 kB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 27.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 84.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.8/56.8 kB 8.1 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 M

In [ ]:
!pip install --upgrade keras tensorflow-addons


# Importing required dependencies
We will import numpy, pandas and re, bertopic, gensim library for now. other libraries will be imported in the notebook later.

Pandas will be used to create a Dataframe and handle the csv file. Numpy will be used for the faster computation of arrays to save time. re library will be used for the cleaning of data. gensim library is used to get coherence score and train LDA. bertopic is used to train bertopic on our Quran-UTM dataset with using pretrained language models Multilingial MiniLM

In [ ]:
import pandas as pd
import numpy as np
import re
from bertopic import BERTopic
from urduhack.normalization import remove_diacritics
from gensim.models import LdaMulticore
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import NMF
from gensim.models.coherencemodel import CoherenceModel
import gensim.corpora as corpora
#optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

##DataFrame




In [ ]:

junapd = pd.read_table("", header=None, encoding='utf-8') #please provide your path

print(junapd.head(5))

stopspd=pd.read_csv('/content/drive/MyDrive/stopwords.txt',names=['List'])

stopspd

                                                   0
0  1|1|(شروع کرتا ہوں) اللہ کے نام سے جو بڑا مہرب...
1  1|2|ہر قسم کی تعریف اس اللہ کے لیے جو سب جہانو...
2  1|3|جو (سب پر) بڑا مہربان (اور خاص بندوں پر) ن...
3          1|4|جزا و سزا کے دن کا مالک (و مختار) ہے۔
4  1|5|(اے اللہ!) ہم تیری ہی عبادت کرتے ہیں اور ت...


,List
0,کی
1,ہیں
2,ہے
3,رہا
4,رہی
...,...
396,گئی
397,ہونے
398,وجہ
399,ہوگیا


## Cleaning of Data
After collecting the eight Quran Urdu translations, we observed these translation had some irrelevant information, such as metadata, punctuation, and diacritics. So we cleaned the translations for topic modeling

Stopwords are common words that are often filtered out during text processing in natural language processing (NLP) tasks. These words are considered to have little or no value in conveying the actual meaning of the text. We take list of 401 stopwords for topic modelling. Stopwords are removed in post preprocessing phase

In [ ]:
import re
#Remove Urdu Punctuation
def remove_urdu_punctuation(text):
    pattern = r'[؛؟،٫٬‘’“”«»!"٪&\'\*\+,-./:;<=>؟@^_`()[]{|}~]'

    cleaned_text = re.sub(pattern, '', text)

    return cleaned_text


In [ ]:
# Remove trailing metadata lines
index_to_drop = junapd[junapd[0].str.startswith("#")].index
junapd.drop(index_to_drop, inplace=True)

# Remove verse reference from every line
junapd[0] = junapd[0].str.replace(r'\d+\|\d+\|', '', regex=True)

# Remove diacritics
junapd[0] = junapd[0].apply(remove_diacritics)

# Remove punctuation
junapd[0] = junapd[0].apply(remove_urdu_punctuation)

# Reset index
junapd.reset_index(drop=True, inplace=True)

# Display the cleaned DataFrame
print("Cleaned DataFrame:")
print(junapd.head())



Cleaned DataFrame:
                                                   0
0  (شروع کرتا ہوں) اللہ کے نام سے جو بڑا مہربان ن...
1  ہر قسم کی تعریف اس اللہ کے لیے جو سب جہانوں کا...
2  جو (سب پر) بڑا مہربان (اور خاص بندوں پر) نہایت...
3              جزا و سزا کے دن کا مالک (و مختار) ہے۔
4  (اے اللہ!) ہم تیری ہی عبادت کرتے ہیں اور تجھ ہ...


In [ ]:
def remove_nonbreaking_space(text):
    return re.sub(r'\xa0', ' ', text)

junapd[0] = junapd[0].apply(remove_nonbreaking_space)

In [ ]:
# Save the cleaned DataFrame to a text file
file_path = 'cleaned_Najafi_text.txt'
junapd[0].to_csv(file_path, sep='\n', index=False, header=False, encoding='utf-8')

print(f'Text has been saved to {file_path}')

Text has been saved to cleaned_Najafi_text.txt


In [ ]:
# Convert the DataFrame to a list of strings
data = junapd[0].tolist()

In [ ]:
print(len(data))

6236


In [ ]:
print(data[56:90])

['اور (وہ وقت یاد کرو) جب ہم نے تمہارے لئے سمندر کو شگافتہ کیا (اسے پھاڑ کر تمہارے لئے راستہ بنایا) اور تمہیں نجات دی اور تمہارے دیکھتے دیکھتے فرعونیوں کو غرق کر دیا۔', 'اور (وہ وقت یاد کرو) جب ہم نے موسی سے (توراۃ دینے کے لئے) چالیس راتوں کا وعدہ کیا تھا پھر تم نے ان کے بعد گو سالہ (بچھڑے) کو (معبود) بنا لیا۔ جبکہ تم ظالم تھے۔', 'پھر ہم نے اس (ظلم) کے بعد بھی معاف کر دیا۔ تاکہ تم شکر گزار بنو۔', 'اور (وہ وقت یاد کرو) جب فرعونی (تم پر ظلم کر رہے تھے تو) ہم نے موسی کو کتاب فرقان عطا کی تاکہ تم ہدایت حاصل کرو۔', 'اور (وہ وقت یاد کرو) جب (موسی نے اپنی قوم سے کہا) اے میری قوم! یقینا تم نے گو سالہ کو (معبود) بنا کر اپنی جانوں پر بڑا ظلم کیا ہے۔ لہذا تم اپنے خالق کی بارگاہ میں (اس طرح) توبہ کرو۔ کہ اپنی جانوں کو قتل کرو۔ یہی (طریقہ کار) تمہارے خالق کے نزدیک تمہارے لئے بہتر ہے۔ اس صورت میں اس نے تمہاری توبہ قبول کی۔ بے شک وہ بڑا توبہ قبول کرنے والا، نہایت رحم والا ہے۔', 'اور (وہ وقت یاد کرو) جب تم نے کہا اے موسی! اس وقت تک ہم ہرگز ایمان نہیں لائیں گے جب تک ظاہر بظاہر (اعلانیہ) خدا کو دیکھ نہ 

# BERTopic Training
The default  bertopic embedding model is paraphrase-multilingual-MiniLM-L12-v2 when selecting language="multilingual". We take MiniLM Model from [sentence-tranformer](https://www.sbert.net/docs/pretrained_models.html) and create custom document embedding and passed it to the bertopic model for training.

In [ ]:
#create custom embedding
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')
embeddings = model.encode(data, show_progress_bar=True)
print(embeddings)

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/4.12k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/645 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/195 [00:00<?, ?it/s]

[[ 0.06645682  0.6879264  -0.22863501 ...  0.02085426  0.23922494
   0.14096878]
 [ 0.0379447   0.6305888  -0.15574239 ... -0.0666512   0.09301648
   0.03851425]
 [-0.06785567  0.2890461  -0.10935669 ... -0.21950457  0.40148735
   0.03132743]
 ...
 [ 0.14954443  0.18782392 -0.00903778 ... -0.02975448  0.27802932
   0.05299997]
 [-0.02709399 -0.01598123 -0.2069522  ...  0.00524382  0.16166873
   0.0694894 ]
 [ 0.06655871  0.09454026 -0.03136014 ... -0.16386788  0.13934168
  -0.20365198]]


In [ ]:
stop_words_list = stopspd['List'].tolist()

In [ ]:
#pass vectorizer_model to bertopic for stopwords removal
from sklearn.feature_extraction.text import CountVectorizer

vectorizer_model = CountVectorizer(stop_words= stop_words_list)


In [ ]:
#UMAP for dimention reduction
from umap import UMAP
dim_model = UMAP(n_components=4, random_state=42)

In [ ]:
# # #KMeans used for clustering
from sklearn.cluster import KMeans

cluster_model = KMeans(n_clusters=5, random_state=42)

In [ ]:
np.random.seed(42)

In [ ]:
topic_model = BERTopic(language="urdu", low_memory=True ,calculate_probabilities=True,  top_n_words=10,hdbscan_model=cluster_model, umap_model=dim_model, verbose=True, vectorizer_model=vectorizer_model)

In [ ]:
#Fit documents in bertopic
topics, probs = topic_model.fit_transform(data,embeddings)

2024-06-09 10:02:57,976 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2024-06-09 10:03:35,292 - BERTopic - Dimensionality - Completed ✓
2024-06-09 10:03:35,294 - BERTopic - Cluster - Start clustering the reduced embeddings
2024-06-09 10:03:35,422 - BERTopic - Cluster - Completed ✓
2024-06-09 10:03:35,431 - BERTopic - Representation - Extracting topics from clusters using representation models.
2024-06-09 10:03:35,597 - BERTopic - Representation - Completed ✓


In [ ]:
print(probs)

None


In [ ]:
#topics that assign to each document
print(topics)

[2, 2, 1, 1, 2, 0, 1, 0, 1, 1, 1, 1, 1, 1, 2, 1, 2, 0, 0, 1, 2, 2, 0, 2, 0, 0, 2, 1, 2, 2, 0, 0, 2, 2, 2, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 2, 0, 2, 0, 0, 1, 1, 1, 2, 1, 2, 2, 2, 2, 2, 2, 0, 2, 1, 0, 1, 1, 1, 0, 0, 1, 2, 2, 2, 2, 1, 1, 1, 0, 2, 1, 2, 1, 0, 2, 1, 1, 0, 2, 2, 2, 0, 2, 2, 1, 1, 1, 2, 2, 2, 1, 2, 1, 2, 1, 1, 0, 1, 1, 2, 1, 2, 2, 1, 2, 2, 2, 0, 1, 2, 2, 2, 2, 2, 0, 2, 2, 1, 1, 1, 1, 2, 2, 2, 2, 0, 2, 2, 0, 2, 1, 2, 2, 1, 2, 0, 1, 1, 2, 1, 1, 0, 1, 1, 1, 2, 2, 1, 1, 1, 1, 1, 0, 0, 1, 2, 0, 0, 2, 0, 2, 0, 0, 2, 2, 2, 2, 2, 2, 0, 2, 2, 1, 2, 1, 2, 2, 0, 1, 1, 2, 1, 2, 2, 1, 1, 1, 2, 2, 2, 2, 2, 1, 1, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 2, 2, 1, 1, 1, 1, 2, 2, 1, 1, 1, 1, 1, 2, 1, 2, 1, 2, 1, 2, 1, 1, 1, 2, 0, 0, 0, 1, 2, 1, 2, 2, 2, 2, 1, 1, 2, 1, 0, 0, 0, 2, 2, 1, 1, 1, 1, 0, 2, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 1, 1, 1, 1, 2, 2, 2, 2, 0, 1, 0, 0, 1, 0, 1, 1, 1, 2, 1, 2, 1, 1, 1, 1, 1, 1, 1, 

In [ ]:
topic_model.get_topic_freq()

,Topic,Count
2,0,2441
1,1,2133
0,2,1623
4,3,31
3,4,8


In [ ]:
document_topics = topic_model.get_topics()

In [ ]:
#topics with score
print(document_topics)

{0: [('دن', 0.03909670217943283), ('کوئی', 0.03892752075602089), ('لوگ', 0.0375502515454563), ('عذاب', 0.030489714973280388), ('لوگوں', 0.027525028901550248), ('تمہیں', 0.027221665085836056), ('جائیں', 0.026277379912864822), ('والوں', 0.024650742966787133), ('پس', 0.022771153687544513), ('تمہارے', 0.02214054396118309)], 1: [('پروردگار', 0.07074108299256163), ('خدا', 0.04393916497034338), ('اے', 0.03987187513089033), ('لوگوں', 0.03385297800437097), ('لوگ', 0.032004893154810796), ('ایمان', 0.03157891142539876), ('والا', 0.029937351898707144), ('کوئی', 0.029422777830213947), ('رسول', 0.028064114454388576), ('تمہارے', 0.027463787124891)], 2: [('اللہ', 0.13825309067178707), ('والا', 0.0454207138261407), ('کوئی', 0.04039178583448148), ('بڑا', 0.0363055389439431), ('لوگ', 0.03561931340176746), ('ایمان', 0.034216805621702415), ('تمہارے', 0.03194155395753991), ('اے', 0.029572714445361612), ('تمہیں', 0.029548368071878942), ('بے', 0.028749392099856486)], 3: [('کن', 1.7005521191170225), ('نعمتوں',

In [ ]:
topic_model.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,0,2441,0_دن_کوئی_لوگ_عذاب,"[دن, کوئی, لوگ, عذاب, لوگوں, تمہیں, جائیں, وال...",[اور ہم نے آسمان و زمین کو اور جو کچھ ان کے در...
1,1,2133,1_پروردگار_خدا_اے_لوگوں,"[پروردگار, خدا, اے, لوگوں, لوگ, ایمان, والا, ک...",[کہا اے میرے پروردگار میں نے اپنے اوپر ظلم کیا...
2,2,1623,2_اللہ_والا_کوئی_بڑا,"[اللہ, والا, کوئی, بڑا, لوگ, ایمان, تمہارے, اے...",[یا اللہ کے سوا ان کا کوئی اور خدا ہے؟ پاک ہے ...
3,3,31,3_کن_نعمتوں_جھٹلاؤ_پروردگار,"[کن, نعمتوں, جھٹلاؤ, پروردگار, پس, انس, جھٹلاؤ...",[پس تم اپنے پروردگار کی کن کن نعمتوں کو جھٹلاؤ...
4,4,8,4_اطاعت_ڈرو_میری_سو,"[اطاعت, ڈرو, میری, سو, اللہ, پس, , , , ]","[سو تم اللہ سے ڈرو اور میری اطاعت کرو۔, سو تم ..."


In [ ]:
topic_distr, _ = topic_model.approximate_distribution(data, window=3, min_similarity=0.01)

100%|██████████| 7/7 [00:01<00:00,  4.30it/s]


In [ ]:
print(topic_distr)

[[0.08154626 0.20583734 0.56761531 0.         0.14500109]
 [0.05468435 0.2507351  0.4575759  0.08863331 0.14837134]
 [0.20885586 0.373798   0.41734614 0.         0.        ]
 ...
 [0.61186163 0.16039896 0.22773941 0.         0.        ]
 [0.30391439 0.3647343  0.3313513  0.         0.        ]
 [0.         0.6324851  0.3675149  0.         0.        ]]


In [ ]:
topic_model.visualize_distribution(topic_distr[0], width=600,height=600, title="Topic Probability Distributio")

# Evaluation
we used three evaluation metrics to compare the results.

1. The coherence score is used to capture the degree of similarity between the words within each topic, with higher scores indicating more coherent topics. We used two coherence metrics NPMI and Cv Score.
2. IRBO measures are used to assess how different and distinct the topics are in a topic model.


### Coherence Score
To evaluate the model topics coherence we use [Gensim](https://radimrehurek.com/gensim/models/coherencemodel.html) library

In [ ]:
topic_model.update_topics(data, vectorizer_model=vectorizer_model)

In [ ]:
texts = [[word for word in str(document).split() if word not in stop_words_list] for document in data] #if word not in stop_words_list
id2word = corpora.Dictionary(texts)
corpus = [id2word.doc2bow(text) for text in texts]

In [ ]:
topics_bert=[]
for i in topic_model.get_topics():
  row=[]
  topic= topic_model.get_topic(i)
  for word in topic:
     row.append(word[0])
  topics_bert.append(row)

In [ ]:
print(topics_bert)

[['دن', 'کوئی', 'لوگ', 'عذاب', 'لوگوں', 'تمہیں', 'جائیں', 'والوں', 'پس', 'تمہارے'], ['پروردگار', 'خدا', 'اے', 'لوگوں', 'لوگ', 'ایمان', 'والا', 'کوئی', 'رسول', 'تمہارے'], ['اللہ', 'والا', 'کوئی', 'بڑا', 'لوگ', 'ایمان', 'تمہارے', 'اے', 'تمہیں', 'بے'], ['کن', 'نعمتوں', 'جھٹلاؤ', 'پروردگار', 'پس', 'انس', 'جھٹلاؤگے', 'سو', 'اے', ''], ['اطاعت', 'ڈرو', 'میری', 'سو', 'اللہ', 'پس', '', '', '', '']]


In [ ]:
# compute Coherence Score CV

cm = CoherenceModel(topics=topics_bert, texts=texts, dictionary=id2word, coherence='c_v')
coherence = round(cm.get_coherence(),2)
print('\nCV Score: ', coherence)


CV Score:  0.55


In [ ]:
# compute Coherence Score NPMI

cm = CoherenceModel(topics=topics_bert, texts=texts, dictionary=id2word, coherence='c_npmi')
coherence = round(cm.get_coherence(),2)
print('\nNPMI Score: ', coherence)


NPMI Score:  0.02


**Diversity Score**

Upload rbo.py file

In [ ]:
import itertools
from rbo import rbo
import numpy as np

class InvertedRBO:
    def __init__(self):
        pass

    def irbo(self, topics, topk=10, weight=0.9):
        """
        Calculate inverted Rank Biased Overlap (RBO) as a measure of topic diversity from a list of lists of words.

        :param topics: A list of lists of words representing different topics.
        :param topk: The number of top words on which RBO will be computed.
        :param weight: Weight of each agreement at depth d: p**(d-1). When set to 1.0, there is no weight,
                       and the RBO returns to average overlap.
        :return: The inverted RBO topic diversity score.
        """
        if topk <= 0:
            raise ValueError("topk must be a positive integer.")

        num_topics = len(topics)
        if num_topics == 0:
            raise ValueError("topics list cannot be empty.")

        if topk > len(topics[0]):
            raise Exception('Words in topics are less than topk')

        collect = []
        for list1, list2 in itertools.combinations(topics, 2):
            rbo_val = rbo(list1[:topk], list2[:topk], p=weight)[2]
            collect.append(rbo_val)

        Irbo_score = 1 - np.mean(collect)
        return Irbo_score

In [ ]:
inverted_rbo_calculator = InvertedRBO()
IRBO= round(inverted_rbo_calculator.irbo(topics_bert, topk=10, weight=0.9),2)
print("Inverted RBO Score:", IRBO)

Inverted RBO Score: 0.86


# Visualize Topics

In [ ]:
topic_model.visualize_topics()

In [ ]:
topic_model.visualize_barchart(n_words=10,width=220, height=270, title="Topic Word Scores")

# Model serialization

In [ ]:
# Save model
topic_model.save("my_model")

2024-06-09 10:06:38,027 - BERTopic - WARNING: When you use `pickle` to save/load a BERTopic model,please make sure that the environments in which you saveand load the model are **exactly** the same. The version of BERTopic,its dependencies, and python need to remain the same.


In [ ]:
loaded_topic_model = topic_model.load("my_model")
loaded_topic_model.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,0,2441,0_دن_کوئی_لوگ_عذاب,"[دن, کوئی, لوگ, عذاب, لوگوں, تمہیں, جائیں, وال...",[اور ہم نے آسمان و زمین کو اور جو کچھ ان کے در...
1,1,2133,1_پروردگار_خدا_اے_لوگوں,"[پروردگار, خدا, اے, لوگوں, لوگ, ایمان, والا, ک...",[کہا اے میرے پروردگار میں نے اپنے اوپر ظلم کیا...
2,2,1623,2_اللہ_والا_کوئی_بڑا,"[اللہ, والا, کوئی, بڑا, لوگ, ایمان, تمہارے, اے...",[یا اللہ کے سوا ان کا کوئی اور خدا ہے؟ پاک ہے ...
3,3,31,3_کن_نعمتوں_جھٹلاؤ_پروردگار,"[کن, نعمتوں, جھٹلاؤ, پروردگار, پس, انس, جھٹلاؤ...",[پس تم اپنے پروردگار کی کن کن نعمتوں کو جھٹلاؤ...
4,4,8,4_اطاعت_ڈرو_میری_سو,"[اطاعت, ڈرو, میری, سو, اللہ, پس, , , , ]","[سو تم اللہ سے ڈرو اور میری اطاعت کرو۔, سو تم ..."


# LDA

We use the [ parallelized Latent Dirichlet Allocation (LDA)](https://radimrehurek.com/gensim/models/ldamulticore.html) from Gensim.

Note: for LDA you have to define topics number in advance.

In [ ]:
n_topics=5
lda = LdaMulticore(corpus, id2word=id2word, random_state=42, num_topics=5)
topics = lda.show_topics(num_topics=5, formatted=False)

#Extract the words from the topics
topics_list = []
for _, topic_words in topics:
    words = [word for word, _ in topic_words]
    topics_list.append(words)

print(topics_list)

In [ ]:
#CV Score
cm = CoherenceModel(topics=topics_list, texts=texts, corpus=corpus, dictionary=id2word, coherence='c_v')
coherence_lda = round(cm.get_coherence(),2)
print('\nCV Score: ', coherence_lda)


CV Score:  0.43


In [ ]:
#NPMI Score
cm = CoherenceModel(topics=topics_list, texts=texts, corpus=corpus, dictionary=id2word, coherence='c_npmi')
coherence_lda = round(cm.get_coherence(),2)
print('\nNPMI Score: ', coherence_lda)


NPMI Score:  -0.01


In [ ]:
# Calculate IRBO Score
inverted_rbo_calculator = InvertedRBO()
IRBO_LDA = round(inverted_rbo_calculator.irbo(topics_list, topk=10, weight=0.9),2)
print('\nIRBO Score: ',IRBO_LDA)


IRBO Score:  0.49


#NMF
We use Gensim library for implementation of NMF

Note: for NMF you have to define topics number in advance.

In [ ]:
#Using Gensim
from gensim import corpora, models


# Train NMF model
num_topics = 5  # Define the number of topics
nmf_model = models.Nmf(corpus, num_topics=num_topics, id2word=id2word, random_state=42)

# Extract topics
topics = []
for topic_id in range(num_topics):
    topic_words = nmf_model.show_topic(topic_id, topn=10)
    topic_words = [word for word, _ in topic_words]
    topics.append(topic_words)

# Print topics in the desired format
print(topics)

In [ ]:
#Calculate CV Score
cm = CoherenceModel(topics=topics, texts=texts, corpus=corpus, dictionary=id2word, coherence='c_v')
coherence_nmf = round(cm.get_coherence(),2)
print('\nCV Score: ', coherence_nmf)

In [ ]:
#Calculate NPMI Score
cm = CoherenceModel(topics=topics, texts=texts, corpus=corpus, dictionary=id2word, coherence='c_npmi')
coherence_nmf = round(cm.get_coherence(),2)
print('\nNPMI Score: ', coherence_nmf)

In [ ]:
# Calculate IRBO Score
inverted_rbo_calculator = InvertedRBO()
IRBO_NMF = round(inverted_rbo_calculator.irbo(topics, topk=10, weight=0.9),2)c
print('\nIRBO Score: ',IRBO_NMF)